In [5]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [2]:
import torch
from torch import nn
from torch import optim

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline

In [10]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [6]:
# neg run 14
n_RQS_knots = 10   # Number of knots in RQS transform
n_made_layers = 3  # Number of layers in every made network
n_made_units = 500 # Number of units in every layer of the made network
n_flow_layers = 12 # Number of layers in the flow

# ### Training procedure parameters

# In[2]:


batch_size = 1024
n_epochs = 800
adam_lr = 0.001   # Learning rate for the ADAM optimizer (default: 0.001)

In [7]:
n_testing = int(3.6820486488184496*1e5)

In [9]:
runname = "neg_-_weights_-_run_{}".format(1)

In [16]:
training_samples = np.genfromtxt("neg_training_samples.csv",
                    delimiter=',').astype(np.float32)
testing_samples = np.genfromtxt("neg_testing_samples.csv",
                    delimiter=',').astype(np.float32)[:n_testing]

OSError: neg_training_samples.csv not found.

In [ ]:
training_weights = np.genfromtxt("/neg_training_weights{}.csv".format(weight_set),
                    delimiter=',').astype(np.float32)
testing_weights = np.genfromtxt("/neg_testing_weights{}.csv".format(weight_set),
                    delimiter=',').astype(np.float32)[:n_testing]

In [ ]:
training_weights = np.sign(training_weights)
testing_weights = np.sign(testing_weights)

In [ ]:
event_dim = training_samples.shape[1] # Dimensionality of data